# Section 2.2: Multivariable Calculus for Dynamic Systems - Exercises

This notebook contains practical exercises for applying multivariable calculus concepts to dynamic systems modeling. We'll explore gradient fields, vector fields, phase portraits, and other essential tools for understanding complex system behavior.

## Learning Objectives
- Apply partial derivatives to analyze system behavior
- Visualize gradient fields and understand their significance
- Analyze vector fields and their relationship to dynamic systems
- Calculate divergence and curl to characterize field properties
- Implement numerical methods for multivariable systems

## Setup

Let's first import the necessary libraries for our exercises.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import sympy as sp
from sympy.utilities.lambdify import lambdify

# Set plotting parameters for better visualization
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12
plt.style.use('ggplot')

# Exercise 1: Vector Fields and Gradient Analysis (Solved)

In this exercise, we'll analyze a potential function and its gradient field, which is fundamental to understanding many dynamic systems including physical, ecological, and economic models.

## Problem Statement

Consider the potential function:

$$f(x, y) = x^2 + y^2 - 2xy$$

This function could represent an energy landscape in a physical system, a fitness landscape in evolutionary biology, or a utility function in economics.

We'll:
1. Compute the gradient field of this function
2. Visualize both the function and its gradient field
3. Find and classify critical points
4. Interpret the dynamic implications

### Step 1: Define the function and compute its gradient symbolically

In [ ]:
# Define symbolic variables
x, y = sp.symbols('x y')

# Define the function
f = x**2 + y**2 - 2*x*y
print(f"Function f(x,y) = {f}")

# Compute the gradient
grad_f_x = sp.diff(f, x)
grad_f_y = sp.diff(f, y)
grad_f = (grad_f_x, grad_f_y)

print(f"Gradient of f: (∂f/∂x, ∂f/∂y) = ({grad_f_x}, {grad_f_y})")

### Step 2: Visualize the potential function as a 3D surface

In [ ]:
# Convert symbolic function to numerical function
f_numeric = lambdify((x, y), f, 'numpy')

# Create a grid of points
x_vals = np.linspace(-3, 3, 100)
y_vals = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x_vals, y_vals)
Z = f_numeric(X, Y)

# Create 3D plot
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, Y, Z, cmap=cm.viridis, alpha=0.8)

# Add contour plot at the bottom
ax.contour(X, Y, Z, zdir='z', offset=np.min(Z), cmap=cm.viridis, alpha=0.5)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f(x,y)')
ax.set_title('Potential Function f(x,y) = x² + y² - 2xy')
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

plt.show()

### Step 3: Visualize the gradient field

The gradient field shows the direction of steepest ascent at each point. If we think of a particle moving on this landscape, it would tend to move in the opposite direction of the gradient (downhill) if seeking to minimize energy.

In [ ]:
# Convert gradient to numerical functions
grad_f_x_numeric = lambdify((x, y), grad_f_x, 'numpy')
grad_f_y_numeric = lambdify((x, y), grad_f_y, 'numpy')

# Create a coarser grid for the vector field
x_grid = np.linspace(-3, 3, 20)
y_grid = np.linspace(-3, 3, 20)
X_grid, Y_grid = np.meshgrid(x_grid, y_grid)
U = grad_f_x_numeric(X_grid, Y_grid)
V = grad_f_y_numeric(X_grid, Y_grid)

# Calculate the magnitude of the gradient for color mapping
gradient_magnitude = np.sqrt(U**2 + V**2)

# Normalize the vectors for better visualization
U_norm = U / (gradient_magnitude + 1e-10)  # Add small value to avoid division by zero
V_norm = V / (gradient_magnitude + 1e-10)

# Create the plot
fig, ax = plt.subplots(figsize=(10, 8))

# Plot contours of the function
contour = ax.contourf(X, Y, Z, levels=20, cmap='viridis', alpha=0.6)

# Plot gradient vectors
quiver = ax.quiver(X_grid, Y_grid, U_norm, V_norm, gradient_magnitude,
                   angles='xy', scale_units='xy', scale=25, cmap='autumn')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Gradient Field of f(x,y) = x² + y² - 2xy with Contours')
fig.colorbar(contour, ax=ax, label='Function Value')
fig.colorbar(quiver, ax=ax, label='Gradient Magnitude', orientation='horizontal', pad=0.1)

plt.tight_layout()
plt.show()

### Step 4: Find and classify critical points

Critical points occur where the gradient is zero. We can find them by solving the system of equations:

$$\frac{\partial f}{\partial x} = 0, \frac{\partial f}{\partial y} = 0$$

In [ ]:
# Find critical points by solving the system grad_f = (0, 0)
critical_points = sp.solve((grad_f_x, grad_f_y), (x, y))
print(f"Critical points: {critical_points}")

# Compute the Hessian matrix to classify the critical points
H_xx = sp.diff(grad_f_x, x)
H_xy = sp.diff(grad_f_x, y)
H_yx = sp.diff(grad_f_y, x)
H_yy = sp.diff(grad_f_y, y)

print(f"Hessian matrix: [({H_xx}, {H_xy}), ({H_yx}, {H_yy})]")

# For each critical point, compute the determinant and trace of the Hessian
for point in critical_points:
    x_val, y_val = point
    
    # Substitute values to evaluate Hessian at this point
    hxx_val = H_xx.subs([(x, x_val), (y, y_val)])
    hxy_val = H_xy.subs([(x, x_val), (y, y_val)])
    hyx_val = H_yx.subs([(x, x_val), (y, y_val)])
    hyy_val = H_yy.subs([(x, x_val), (y, y_val)])
    
    # Calculate determinant and trace
    det_H = hxx_val * hyy_val - hxy_val * hyx_val
    trace_H = hxx_val + hyy_val
    
    # Classify the critical point
    if det_H > 0:
        if trace_H > 0:
            point_type = "Minimum"
        else:
            point_type = "Maximum"
    elif det_H < 0:
        point_type = "Saddle point"
    else:
        point_type = "Degenerate critical point"
    
    # Function value at the critical point
    f_val = f.subs([(x, x_val), (y, y_val)])
    
    print(f"Point ({x_val}, {y_val}) is a {point_type} with f(x,y) = {f_val}")
    print(f"  Determinant of Hessian: {det_H}")
    print(f"  Trace of Hessian: {trace_H}\n")

### Step 5: Visualize dynamical implications

In many systems, particles or agents tend to move along the negative gradient direction (downhill). Let's simulate several trajectories starting from different initial conditions.

In [ ]:
# Define gradient descent function
def gradient_descent(start_point, learning_rate=0.1, num_steps=100):
    trajectory = np.zeros((num_steps, 2))
    trajectory[0] = start_point
    
    for i in range(1, num_steps):
        x_current, y_current = trajectory[i-1]
        grad_x = grad_f_x_numeric(x_current, y_current)
        grad_y = grad_f_y_numeric(x_current, y_current)
        
        # Move in the negative gradient direction
        x_new = x_current - learning_rate * grad_x
        y_new = y_current - learning_rate * grad_y
        
        trajectory[i] = [x_new, y_new]
    
    return trajectory

# Generate some random starting points
np.random.seed(42)
num_trajectories = 8
start_points = 5 * np.random.rand(num_trajectories, 2) - 2.5  # Random points in [-2.5, 2.5]^2

# Plot the contour map with trajectories
fig, ax = plt.subplots(figsize=(10, 8))

# Plot contours of the function
contour = ax.contourf(X, Y, Z, levels=20, cmap='viridis', alpha=0.7)

# Plot critical points
for point in critical_points:
    x_val, y_val = point
    ax.plot(x_val, y_val, 'ro', markersize=10)
    ax.text(x_val + 0.1, y_val + 0.1, f'({x_val}, {y_val})')

# Compute and plot each trajectory
for i, start in enumerate(start_points):
    trajectory = gradient_descent(start, learning_rate=0.05, num_steps=100)
    ax.plot(trajectory[:, 0], trajectory[:, 1], 'k-', linewidth=1, alpha=0.8)
    ax.plot(trajectory[0, 0], trajectory[0, 1], 'go', markersize=5)
    ax.text(trajectory[0, 0] + 0.1, trajectory[0, 1] + 0.1, f'Start {i+1}')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Gradient Descent Trajectories')
fig.colorbar(contour, ax=ax, label='Function Value')

plt.tight_layout()
plt.show()

### Analysis and Interpretation

Our analysis of the function $f(x, y) = x^2 + y^2 - 2xy$ reveals several important insights:

1. **Critical Points**: We found one critical point at (0, 0), which is a minimum. This is the equilibrium point of the system.

2. **Gradient Field**: The gradient vectors point radially outward from the minimum, showing the direction of steepest increase.

3. **Dynamic Behavior**: The gradient descent trajectories show how particles or agents in this system would naturally move toward the minimum at (0, 0), regardless of where they start (within the basin of attraction).

4. **Physical Interpretation**: If this were an energy landscape, objects would naturally roll downhill toward the minimum energy state at (0, 0). In an ecological context, this might represent evolution toward an optimal trait combination.

5. **System Stability**: The positive determinant and trace of the Hessian at the critical point confirm it's a stable equilibrium (minimum), meaning small perturbations will result in forces that push the system back toward equilibrium.

This type of analysis is fundamental to understanding the qualitative behavior of many complex systems, from mechanical systems to population dynamics and economic models.

# Exercise 2: Vector Field Analysis and Flow Visualization (Unsolved)

## Problem Statement

Consider the following vector field representing a dynamical system:

$$\vec{F}(x, y) = (y, -\sin(x))$$

This type of vector field appears in many physical and biological systems, including pendulum dynamics, predator-prey models, and certain types of electrical circuits.

Your tasks are:

1. Visualize the vector field $\vec{F}(x, y)$
2. Calculate and interpret the divergence and curl of the field
3. Determine if the field is conservative by checking if it can be written as the gradient of some potential function
4. Simulate and visualize the flow of particles in this field (particle trajectories)
5. Find and classify the equilibrium points of the system
6. Analyze the system's behavior and stability properties

Use the concepts from Section 2.2 of the course to perform this analysis.

### Setup: Define the vector field

In [ ]:
# Define symbolic variables
x, y = sp.symbols('x y')

# Define the vector field components
F_x = y
F_y = -sp.sin(x)

print(f"Vector field F(x,y) = ({F_x}, {F_y})")

# Convert to numerical functions for plotting
F_x_numeric = lambdify((x, y), F_x, 'numpy')
F_y_numeric = lambdify((x, y), F_y, 'numpy')

### Step 1: Visualize the vector field

Complete this step by visualizing the vector field using quiver plot. You'll need to:
- Create a grid of points
- Evaluate the vector field at each point
- Create a quiver plot with appropriate scaling and coloring

In [ ]:
# Your code for visualizing the vector field goes here
# Hint: Use plt.quiver() for creating a vector field plot

# Create a grid of points
# ...

# Evaluate the vector field at each point
# ...

# Calculate the magnitude for coloring
# ...

# Create the quiver plot
# ...

### Step 2: Calculate and interpret the divergence and curl

Calculate the divergence and curl of the vector field, then interpret what these values mean for the behavior of the system.

Remember:
- Divergence: $\nabla \cdot \vec{F} = \frac{\partial F_x}{\partial x} + \frac{\partial F_y}{\partial y}$
- Curl: $\nabla \times \vec{F} = \frac{\partial F_y}{\partial x} - \frac{\partial F_x}{\partial y}$ (for 2D fields)

In [ ]:
# Your code for calculating divergence and curl goes here
# Hint: Use symbolic differentiation with sympy

# Calculate divergence
# ...

# Calculate curl (in 2D, this is a scalar)
# ...

# Print and interpret the results
# ...

### Step 3: Determine if the field is conservative

A vector field is conservative if it can be written as the gradient of some scalar potential function. This happens if and only if the curl is zero everywhere.

If the field is conservative, find the potential function $\phi$ such that $\vec{F} = \nabla \phi$.

In [ ]:
# Your code for determining if the field is conservative goes here
# If conservative, find the potential function
# Hint: Use the condition that curl = 0 everywhere for conservative fields

# Check if the curl is zero everywhere
# ...

# If conservative, find the potential function by integrating
# ...

### Step 4: Simulate particle trajectories

Implement a numerical integrator (like Euler or Runge-Kutta) to simulate the paths that particles would follow in this vector field. These paths are governed by the differential equations:

$$\frac{dx}{dt} = F_x(x, y) = y$$
$$\frac{dy}{dt} = F_y(x, y) = -\sin(x)$$

In [ ]:
# Your code for simulating particle trajectories goes here
# Hint: Use a numerical integration method like Runge-Kutta

# Define the differential equations
# ...

# Implement a numerical integrator
# ...

# Choose several initial conditions
# ...

# Simulate and plot trajectories
# ...

### Step 5: Find and classify equilibrium points

Equilibrium points occur where the vector field is zero: $\vec{F}(x, y) = (0, 0)$. Find these points and classify them based on the eigenvalues of the Jacobian matrix at each point.

In [ ]:
# Your code for finding and classifying equilibrium points goes here
# Hint: Solve F_x = 0 and F_y = 0 simultaneously

# Find equilibrium points
# ...

# Compute the Jacobian matrix
# ...

# For each equilibrium point, evaluate the Jacobian and find eigenvalues
# ...

# Classify each point (e.g., stable/unstable node, saddle, center, spiral)
# ...

### Step 6: Analyze and interpret the system

Based on your findings, provide a comprehensive analysis of the system's behavior. Include:
- The physical interpretation of the vector field
- The significance of equilibrium points and their stability
- Global behavior patterns and any special trajectories
- Connections to real-world systems that might exhibit similar dynamics

## Your analysis here

Write a comprehensive analysis of the vector field and its dynamics...

Points to consider:
- What do the equilibrium points represent physically?
- How do trajectories behave near and far from equilibrium points?
- What does the divergence and curl tell you about the field?
- Can you recognize any patterns similar to known physical systems?
- Are there any conserved quantities in this system?
- How might external perturbations affect the system behavior?

# Further Exploration

Some ideas for extending your analysis:

1. Modify the vector field slightly and observe how the behavior changes
2. Add damping or forcing terms and analyze the modified system
3. Generate a phase portrait with nullclines to gain additional insight
4. Investigate whether the system has any conserved quantities
5. Relate this system to a specific real-world application

## Recommended Resources

- Strogatz, S. H. (2018). Nonlinear Dynamics and Chaos
- Hirsch, M. W., Smale, S., & Devaney, R. L. (2012). Differential Equations, Dynamical Systems, and an Introduction to Chaos
- Teschl, G. (2012). Ordinary Differential Equations and Dynamical Systems